# Read Images
Notebook to display images and do various test

In [ ]:
import junodch_utils_read_img as utils

import matplotlib.pyplot as matPlt
import numpy as np

import rasterio
from rasterio import plot as rastPlt

## Test display 
Display images from various satellites and extract an areas

In [ ]:
folderName = "img/Test/"
fig, axs = matPlt.subplots(1,4,figsize=(15,15))

axs[0].set_title("Sentinel-2")
with rasterio.open(folderName + "Sentinel-2_1.tif") as r:
    rastPlt.show(r, ax=axs[0])

axs[1].set_xlim(axs[0].get_xlim())
axs[1].set_ylim(axs[0].get_ylim())
axs[2].set_xlim(axs[0].get_xlim())
axs[2].set_ylim(axs[0].get_ylim())
axs[3].set_xlim(axs[0].get_xlim())
axs[3].set_ylim(axs[0].get_ylim())

axs[1].set_title("Landsat-8")
with rasterio.open(folderName + "Landsat-8_1.tif") as r: 
    rastPlt.show(r, ax=axs[1])

axs[2].set_title("VIIRS Nighttime")
with rasterio.open(folderName + "Night VIIRS_1.tif") as r:
    rastPlt.show(r, ax=axs[2])

axs[3].set_title("Human settlement")
with rasterio.open(folderName + "Population GHSL_1.tif") as r:
    rastPlt.show(r, ax=axs[3])


In [ ]:
folderName = "img/Test/"
pathSatellite = folderName + "Sentinel-2_1.tif"
pathNight = folderName + "Night VIIRS_1.tif"
pathValidation = folderName + "Population GHSL_1.tif"

fig, axs = matPlt.subplots(1,2,figsize=(10,10))

aoi = utils.getImgBorder(pathSatellite)

# Fetch coords
dataCoords, dataRadiance = utils.getTilesCoordsPerimeter(pathNight, area=aoi)

lightMask = dataRadiance>250
darkMask = dataRadiance<=250

lightCoords = dataCoords[lightMask]
darkCoords = dataCoords[darkMask]

# Fetch images
with rasterio.open(pathSatellite) as f:
  lightData, lightTransform = utils.getImgFromCoord(f, lightCoords, False)
  darkData, darkTransform = utils.getImgFromCoord(f, darkCoords, False)

  rastPlt.show(lightData, transform=lightTransform, ax=axs[0])
  rastPlt.show(darkData, transform=darkTransform, ax=axs[1])

## Tiles visualization
A look to the tiles on the day satellite images that produce light on the night images.

In [ ]:
folderName = "img/Sokoto/"
pathSatellite = folderName + "Sentinel-2.tif"
pathNight = folderName + "Night_VIIRS_2.tif"
#pathNight = folderName + "lowres_night_1.tif"
pathValidation = folderName + "Population GHSL_1.tif"

aoi = utils.getImgBorder(pathSatellite)

dataCoords, dataRadiance = utils.getTilesCoordsPerimeter(pathNight, area=aoi)

#lightCoords = dataCoords[dataRadiance>200]

print('Tiles:',dataCoords.shape[0])
#print('Light Tile:',lightCoords.shape[0])

In [ ]:
def displayImgsInMask(dataCoords, pathSatellite, mask, batch=100):
  iSave = 0
  iNext = 0
  coords = dataCoords[mask]
  size = coords.shape[0]
  print('Size:',size)

  titles = ['']*size
  for i, v in enumerate(np.argwhere(mask)):
    titles[i] = 'N° ' + str(v)

  with rasterio.open(pathSatellite) as f:
    while iNext < size:
      iNext = iSave+batch
      if iNext > size:
        iNext = size
      dataImgs, _ = utils.getEachImgFromCoord(f, coords[iSave:iNext], True)
      dataImgs = utils.formatData(dataImgs, 32, False)
      
      utils.displayImgs(dataImgs, titles[iSave:iNext])
      iSave = iNext


In [ ]:
print('radiance >= 250:')

displayImgsInMask(dataCoords, pathSatellite, dataRadiance>=250)
print('radiance >= 200:')
displayImgsInMask(dataCoords, pathSatellite, (dataRadiance<250) & (dataRadiance>=200))
print('radiance >= 150:')
displayImgsInMask(dataCoords, pathSatellite, (dataRadiance<200) & (dataRadiance>=150))
print('radiance >= 100:')
displayImgsInMask(dataCoords, pathSatellite, (dataRadiance<150) & (dataRadiance>=100))
#print('radiance >= 75:')
#displayImgsInMask(dataCoords, pathSatellite, (dataRadiance<100) & (dataRadiance>=75))
#print('radiance >= 50:')
#displayImgsInMask(dataCoords, pathSatellite, (dataRadiance<75) & (dataRadiance>=50))
#print('radiance >= 25:')
#displayImgsInMask(dataCoords, pathSatellite, (dataRadiance<50) & (dataRadiance>=25))
#print('radiance > 1:')
#displayImgsInMask(dataCoords, pathSatellite, (dataRadiance<25) & (dataRadiance>1))